In [1]:
pip install datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import os
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, concatenate, LSTM, Conv1D, BatchNormalization, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model, load_model

from datasets import load_dataset
import pandas as pd
import numpy as np
from numpy import zeros
import pyarrow as pa
import sklearn
import string
import nltk

nltk.download('vader_lexicon')
nltk.download('omw-1.4')
nltk.download('wordnet') #synonyms and antonyms
nltk.download('stopwords') #common words
nltk.download('punkt') #tokenizer, divides sentences

from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer #lemmatization
from nltk.tokenize import word_tokenize #splitting sentence into words
from nltk.corpus import stopwords 

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import inflect #generates plurals, convert numbers to words
import gensim #processing texts and vector models
import string
import re


%matplotlib inline
STOPWORDS= set(stopwords.words('english'))
sia= SentimentIntensityAnalyzer()
p=inflect.engine()

In [ ]:
dataset = load_dataset(
    "amazon_reviews_multi", 
    'en' # Select language of the dataset
)

# Coursework


In [ ]:
Train_table=dataset["train"]
Test_table=dataset["test"]
Val_table=dataset["validation"]

In [ ]:
df_train=Train_table.to_pandas()
df_test=Test_table.to_pandas()
df_val=Val_table.to_pandas()

In [ ]:
X_train=df_train.drop(columns=['product_id','reviewer_id','language','product_category'])
X_test=df_test.drop(columns=['review_id','product_id','reviewer_id','language','product_category'])
X_val=df_val.drop(columns=['review_id','product_id','reviewer_id','language','product_category'])

In [ ]:
X_train['rating']=np.where((X_train['stars']<=3),0,X_train.stars)
X_train['rating']=np.where((X_train['stars']>3),1,X_train.rating)
X_val['rating']=np.where((X_val['stars']<=3),0,X_val.stars)
X_val['rating']=np.where((X_val['stars']>3),1,X_val.rating)
X_test['rating']=np.where((X_test['stars']<=3),0,X_test.stars)
X_test['rating']=np.where((X_test['stars']>3),1,X_test.rating)

In [ ]:
X_train['Word_Counts']=X_train['review_body'].apply(lambda x: len(str(x.split())))
X_val['Word_Counts']=X_val['review_body'].apply(lambda x: len(str(x.split())))
X_test['Word_Counts']=X_test['review_body'].apply(lambda x: len(str(x.split())))

# Data Cleaning

In [ ]:
def text_preprocessing(text):
    def remove_emoji(text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F" 
                                   u"\U0001F300-\U0001F5FF"
                                   u"\U0001F680-\U0001F6ff"
                                   u"\U0001F1E0-\U0001F1ff"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+",flags=re.UNICODE)
        return emoji_pattern.sub(r'',text)
    reviews=[text.lower() for text in text]
    reviews=[re.sub(r'\S+@\S+','',text) for text in reviews]
    reviews=[re.sub(r'\d+','',text) for text in reviews]
    reviews=[re.sub(r'[^\w\s]','',text) for text in reviews]
    reviews=[text.strip() for text in reviews]
    reviews=[remove_emoji(text) for text in reviews]
    
    stop_words=set(stopwords.words('english'))
    
    cleaned_reviews=[]
    
    for review in reviews:
        tokens= [word for word in word_tokenize(review) if not word in stop_words]
        cleaned_reviews.append(" ".join(tokens))
        
    lemmatizer = WordNetLemmatizer()
    lem_reviews=[]
    for review in cleaned_reviews:
        lem_reviews.append(" ".join(list(map(lemmatizer.lemmatize,word_tokenize(review)))))
    return lem_reviews

# Preprocessing

In [ ]:
t=X_train['review_body']
X_train['review_body']=text_preprocessing(t)
te=X_test['review_body']
X_test['review_body']=text_preprocessing(te)
tv=X_val['review_body']
X_val['review_body']=text_preprocessing(tv)

In [ ]:
X_train['Word_counts_after_Preprocess']=X_train['review_body'].apply(lambda x: len(str(x.split())))
X_val['Word_counts_after_Preprocess']=X_val['review_body'].apply(lambda x: len(str(x.split())))
X_test['Word_counts_after_Preprocess']=X_test['review_body'].apply(lambda x: len(str(x.split())))


In [ ]:
Y_train=X_train['stars']
X_train.drop('rating',axis=1,inplace=True)

Y_val=X_val['stars']
X_val.drop('rating',axis=1,inplace=True)

Y_test=X_test['stars']
X_test.drop('rating',axis=1,inplace=True)

# VADER

In [ ]:
res1=dict()
for i,row in tqdm(X_train.iterrows(), total=len(X_train)):
    text=row['review_body']
    myid=row['review_id']
    res1[myid]=sia.polarity_scores(text)

In [ ]:
com=pd.DataFrame(res1).T
com=com.reset_index().rename(columns={'index':'review_id'})
com=com.merge(X_train,how='left')

In [ ]:
sns.barplot(data=com,x='stars',y='compound').set(title='Compound Score Plot')

In [ ]:
fig, axs=plt.subplots(1,3,figsize=(18,5))
sns.barplot(data=com,x='stars',y='pos',ax=axs[0]).set(title='Positive Score Plot')
sns.barplot(data=com,x='stars',y='neu',ax=axs[1]).set(title='Neutral Score Plot')
sns.barplot(data=com,x='stars',y='neg',ax=axs[2]).set(title='Negative Score Plot')

# Tokenization

In [ ]:
tokenizer= Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train['review_body'])

X_train['text_token']=tokenizer.texts_to_sequences(X_train['review_body'].values)
X_test['text_token']=tokenizer.texts_to_sequences(X_test['review_body'].values)
X_val['text_token']=tokenizer.texts_to_sequences(X_val['review_body'].values)

# Padding

In [ ]:
max_review_length = 1500
X_train=sequence.pad_sequences(X_train['text_token'].values,maxlen=max_review_length, padding='post')
X_val=sequence.pad_sequences(X_val['text_token'].values,maxlen=max_review_length, padding='post')
X_test=sequence.pad_sequences(X_test['text_token'].values,maxlen=max_review_length, padding='post')

In [ ]:
X_train

# GloVe vector

In [ ]:
g=open(r'./drive/MyDrive/glove.6B.50d.txt',encoding='utf-8')

In [ ]:
glove_vectors=dict()

In [ ]:
for line in g:
    values=line.split()
    word=values[0]
    vectors=np.asarray(values[1:])
    glove_vectors[word]=vectors
g.close()

In [ ]:
glove_words=set(glove_vectors.keys())

In [ ]:
max_vocabulary = len(tokenizer.word_index)
embedding_matrix=zeros((max_vocabulary+1,50))

for word, i in tokenizer.word_index.items():
    if word in glove_words:
        embedding_vector = glove_vectors[word]
        embedding_matrix[i]=embedding_vector

In [ ]:
review = Input(shape=(1500,),name='review_input')

X=Embedding(output_dim=50, input_dim=max_vocabulary+1, input_length=1400, weights=[embedding_matrix],trainable= False)(review)
lstm_review =Bidirectional(LSTM(10))(X)

model= Dropout(0.5)(lstm_review)

model = Flatten()(model)

model= Dense(64,activation= 'relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(model)
model= Dense(8,activation= 'relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.001))(model)
output=Dense(5, activation= 'softmax', name='output')(model)
model= Model(inputs=[review],outputs=[output])

print(model.summary())


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
Y_train = Y_train.to_numpy()
Y_val = Y_val.to_numpy()
Y_test = Y_test.to_numpy()

In [ ]:
Y_train= Y_train.reshape((-1,1))
Y_val = Y_val.reshape((-1,1))
Y_test = Y_test.reshape((-1,1))
Y_train -= 1
Y_val -= 1
Y_test -= 1

In [ ]:
history=model.fit(X_train,Y_train, batch_size=4096, epochs=50, verbose=1,validation_data=(X_val,Y_val))

# Model Save

In [ ]:
model.save('AmazonModel.h5')
model.save_weights('AmazonModel_weights.h5')

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
predictions=model.predict(X_test)
predictions = np.argmax(predictions)
predictions=predictions.round().astype('float')
predictions

In [ ]:
from sklearn.metrics import accuracy_score
Acc=accuracy_score(predictions,Y_test)
print('The accuracy of the model {}'.format(Acc))

# Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(predictions, Y_test)
sns.heatmap(cm,square=True, annot=True, cmap='Blues', fmt='d', cbar= False)